In [ ]:
import numpy as np
import scipy as sp
import scipy.io
import os
import os.path
import PIL.Image
import sys

import keras
from keras.preprocessing import sequence
from keras.layers.core import Masking
from keras.layers import *
from keras.models import Sequential

from keras.utils import to_categorical

from sklearn.utils import shuffle

import matplotlib.pyplot as plt

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'netooarest'
plt.rcParams['image.cmap'] = 'gray'

import pandas as pd
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

In [ ]:
# datasetDir = r'/media/ubuntu/Storage/Proj/trajs/allData/seriesFeatResampled'
datasetDir = r'/home/ubuntu/Documents/traj_mining/data/data_all'

MBMC14list = []

fileList = []
for root,dirs,files in os.walk(datasetDir):
    for file in files:
        fileList.append(os.path.join(root,file))

fileList.sort()

from sklearn.model_selection import train_test_split

traindata, testdata = train_test_split(fileList,test_size=0.2, random_state = 0)


In [ ]:
def loadSeries(filelist,idx):
    filename = filelist[idx]
    name = filename.split('/')[-1].strip('.mat')
    label = filename.split('/')[-2]
    y = label
    data = open(filename, 'r').readlines()[1].replace('\n','').split(':')[1].split(',')
    series = [int(d) for d in data]
    return name, label, MBMC14list.index(y), series


In [ ]:
# traindata  = traindata[:5000]
# testdata = testdata[:2000]
min_length = 5
xs = []
ys = []
for i,file in enumerate(traindata):
    if i%100==0:
        print(i)
    _,_,y,x = loadSeries(traindata,i)
    if len(x)>min_length:
        if (len(x)>300):
            xs.append(x[:300])
        else:
            xs.append(x)
        ys.append(y)
xs = sequence.pad_sequences(xs,maxlen=300,padding='post')

In [ ]:
xs

In [ ]:
xs.shape

In [ ]:
z = sequence.pad_sequences(xs,maxlen=300,padding='post')
ys_ = to_categorical(ys)
ys_.shape

In [ ]:
testxs = []
testys = []
for i,file in enumerate(testdata):
    _,_,y,x = loadSeries(testdata,i)
    if len(x)>min_length:
        if (len(x)>300):
            testxs.append(x[:300])
        else:
            testxs.append(x)
        testys.append(y)    
    if i%100==0:
        print(i)
testxs = sequence.pad_sequences(testxs,maxlen=300,padding='post')
testz = sequence.pad_sequences(testxs,maxlen=300,padding='post')
testys_ = to_categorical(testys)

In [ ]:
testys_.shape, testxs.shape

In [ ]:
model = Sequential()
model.add(Embedding(170000,64,input_length=300))
# model.add(LSTM(64,dropout_W=0.5,dropout_U=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0,return_sequences=True,
              kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros'))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(14,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
modelResult = model.fit(z,ys_,epochs=10,validation_data=(testz,testys_))

In [ ]:
testpredscore = model.predict(testz)

In [ ]:
testpredy = np.argmax(testpredscore,axis=1)

In [ ]:
testpredy.shape

In [ ]:
(testpredy==np.array(testys)).sum() / len(testys)

In [ ]:
from sklearn import metrics
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print("Normalized confusion matrix")
    else:
        # print('Confusion matrix, without normalization')
        pass

    # print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    # plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Compute confusion matrix
cnf_matrix = metrics.confusion_matrix(testys, testpredy)
np.set_printoptions(precision=2)

class_names = MBMC16list
# Plot non-normalized confusion matrix
plt.figure(figsize=[10,10])
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.show()

# Plot normalized confusion matrix
plt.figure(figsize=[10,10])
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, 
                      title='Normalized confusion matrix')
plt.show()